# KNN - Regression

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.neighbors import KNeighborsRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv('Train.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.shape

(8523, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
len(df['Item_Identifier'].unique())

1559

In [7]:
df['Item_Identifier'].value_counts()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

# 1. Item_weight

In [8]:
df['Item_Weight'].fillna(df['Item_Weight'].median(), inplace = True)

# 2. Outlet_Identifier 

In [9]:
df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [10]:
Outlet_Identifier_df = pd.get_dummies(df['Outlet_Identifier'], drop_first = True, prefix = 'Outlet_Identifier' )

# 3. Item_Fat_Content

In [11]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [12]:
df['Item_Fat_Content'].replace({'Low Fat':0,'Regular':1,'LF':0,'reg':1,'low fat':0}, inplace = True)

In [13]:
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228


# 4. Item_Type

In [14]:
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [15]:
Item_Type_df = pd.get_dummies(df['Item_Type'], prefix = 'Item_Type', drop_first = True)
Item_Type_df

,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8519,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8520,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
8521,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [16]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [17]:
df['Outlet_Size'].replace({'Small':0, 'Medium':1, 'High': 2}, inplace = True)
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1.0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,1.0,Tier 3,Supermarket Type2,443.4228


In [18]:
df['Outlet_Size'].fillna(1, inplace = True)
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1.0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,1.0,Tier 3,Supermarket Type2,443.4228


# 5. Outlet_Location_Type

In [19]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [20]:
df['Outlet_Location_Type'].replace({'Tier 1' : 0,'Tier 2':1, 'Tier 3':2}, inplace = True)

In [21]:
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1.0,0,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,1.0,2,Supermarket Type2,443.4228


# 6. Outlet_Type

In [22]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [23]:
Outlet_Type_df = pd.get_dummies(df['Outlet_Type'], prefix = 'Outlet_Type', drop_first = True)
Outlet_Type_df

,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,1,0,0
1,0,1,0
2,1,0,0
3,0,0,0
4,1,0,0
...,...,...,...
8518,1,0,0
8519,1,0,0
8520,1,0,0
8521,0,1,0


In [24]:
df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1.0,0,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,1.0,2,Supermarket Type2,443.4228


In [58]:
data =df
data

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Est_Year
0,9.300,0,0.016047,249.8092,1.0,0,3735.1380,0,0,0,...,0,0,0,0,0,0,1,0,0,23
1,5.920,1,0.019278,48.2692,1.0,2,443.4228,0,0,1,...,0,0,0,0,1,0,0,1,0,13
2,17.500,0,0.016760,141.6180,1.0,0,2097.2700,0,0,0,...,1,0,0,0,0,0,1,0,0,23
3,19.200,1,0.000000,182.0950,1.0,2,732.3800,0,0,0,...,0,0,0,0,0,0,0,0,0,24
4,8.930,0,0.000000,53.8614,2.0,2,994.7052,1,0,0,...,0,0,0,0,0,0,1,0,0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,2.0,2,2778.3834,1,0,0,...,0,0,0,1,0,0,1,0,0,35
8519,8.380,1,0.046982,108.1570,1.0,1,549.2850,0,0,0,...,0,0,0,0,0,0,1,0,0,20
8520,10.600,0,0.035186,85.1224,0.0,1,1193.1136,0,0,0,...,0,0,0,0,0,0,1,0,0,18
8521,7.210,1,0.145221,103.1332,1.0,2,1845.5976,0,0,1,...,0,0,0,1,0,0,0,1,0,13


In [26]:
df.drop(['Outlet_Identifier','Item_Type','Outlet_Type'], axis = 1, inplace = True)

In [27]:
df.drop(['Item_Identifier'], axis =1 , inplace = True)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int64  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_MRP                         8523 non-null   float64
 4   Outlet_Size                      8523 non-null   float64
 5   Outlet_Location_Type             8523 non-null   int64  
 6   Item_Outlet_Sales                8523 non-null   float64
 7   Outlet_Identifier_OUT013         8523 non-null   uint8  
 8   Outlet_Identifier_OUT017         8523 non-null   uint8  
 9   Outlet_Identifier_OUT018         8523 non-null   uint8  
 10  Outlet_Identifier_OUT019         8523 non-null   uint8  
 11  Outlet_Identifier_OUT027         8523 non-null   uint8  
 12  Outlet_Identifier_OU

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Est_Year
0,9.300,0,0.016047,249.8092,1.0,0,3735.1380,0,0,0,...,0,0,0,0,0,0,1,0,0,23
1,5.920,1,0.019278,48.2692,1.0,2,443.4228,0,0,1,...,0,0,0,0,1,0,0,1,0,13
2,17.500,0,0.016760,141.6180,1.0,0,2097.2700,0,0,0,...,1,0,0,0,0,0,1,0,0,23
3,19.200,1,0.000000,182.0950,1.0,2,732.3800,0,0,0,...,0,0,0,0,0,0,0,0,0,24
4,8.930,0,0.000000,53.8614,2.0,2,994.7052,1,0,0,...,0,0,0,0,0,0,1,0,0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,2.0,2,2778.3834,1,0,0,...,0,0,0,1,0,0,1,0,0,35
8519,8.380,1,0.046982,108.1570,1.0,1,549.2850,0,0,0,...,0,0,0,0,0,0,1,0,0,20
8520,10.600,0,0.035186,85.1224,0.0,1,1193.1136,0,0,0,...,0,0,0,0,0,0,1,0,0,18
8521,7.210,1,0.145221,103.1332,1.0,2,1845.5976,0,0,1,...,0,0,0,1,0,0,0,1,0,13


In [29]:
df_list = [df,Outlet_Identifier_df,Item_Type_df,Outlet_Type_df]
df = pd.concat(df_list, axis = 1)
df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,...,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0,0.016047,249.8092,1999,1.0,0,3735.1380,0,0,...,0,0,0,0,0,0,0,1,0,0
1,5.920,1,0.019278,48.2692,2009,1.0,2,443.4228,0,0,...,0,0,0,0,0,1,0,0,1,0
2,17.500,0,0.016760,141.6180,1999,1.0,0,2097.2700,0,0,...,0,1,0,0,0,0,0,1,0,0
3,19.200,1,0.000000,182.0950,1998,1.0,2,732.3800,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.930,0,0.000000,53.8614,1987,2.0,2,994.7052,1,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,1987,2.0,2,2778.3834,1,0,...,0,0,0,0,1,0,0,1,0,0
8519,8.380,1,0.046982,108.1570,2002,1.0,1,549.2850,0,0,...,0,0,0,0,0,0,0,1,0,0
8520,10.600,0,0.035186,85.1224,2004,0.0,1,1193.1136,0,0,...,0,0,0,0,0,0,0,1,0,0
8521,7.210,1,0.145221,103.1332,2009,1.0,2,1845.5976,0,0,...,0,0,0,0,1,0,0,0,1,0


In [30]:
df['Est_Year'] = 2022 - df['Outlet_Establishment_Year']

In [31]:
df['Est_Year']

0       23
1       13
2       23
3       24
4       35
        ..
8518    35
8519    20
8520    18
8521    13
8522    25
Name: Est_Year, Length: 8523, dtype: int64

In [32]:
df.drop(['Outlet_Establishment_Year'], axis = 1, inplace = True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int64  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_MRP                         8523 non-null   float64
 4   Outlet_Size                      8523 non-null   float64
 5   Outlet_Location_Type             8523 non-null   int64  
 6   Item_Outlet_Sales                8523 non-null   float64
 7   Outlet_Identifier_OUT013         8523 non-null   uint8  
 8   Outlet_Identifier_OUT017         8523 non-null   uint8  
 9   Outlet_Identifier_OUT018         8523 non-null   uint8  
 10  Outlet_Identifier_OUT019         8523 non-null   uint8  
 11  Outlet_Identifier_OUT027         8523 non-null   uint8  
 12  Outlet_Identifier_OU

In [34]:
df.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Size', 'Outlet_Location_Type', 'Item_Outlet_Sales',
       'Outlet_Identifier_OUT013', 'Outlet_Identifier_OUT017',
       'Outlet_Identifier_OUT018', 'Outlet_Identifier_OUT019',
       'Outlet_Identifier_OUT027', 'Outlet_Identifier_OUT035',
       'Outlet_Identifier_OUT045', 'Outlet_Identifier_OUT046',
       'Outlet_Identifier_OUT049', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       'Item_Type_Health and Hygiene', 'Item_Type_Household', 'Item_Type_Meat',
       'Item_Type_Others', 'Item_Type_Seafood', 'Item_Type_Snack Foods',
       'Item_Type_Soft Drinks', 'Item_Type_Starchy Foods',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket Type3', 'Est_Year'],
      dtype='object')

In [35]:
'Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP', 'Est_Year'

('Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP', 'Est_Year')

In [36]:
df

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Est_Year
0,9.300,0,0.016047,249.8092,1.0,0,3735.1380,0,0,0,...,0,0,0,0,0,0,1,0,0,23
1,5.920,1,0.019278,48.2692,1.0,2,443.4228,0,0,1,...,0,0,0,0,1,0,0,1,0,13
2,17.500,0,0.016760,141.6180,1.0,0,2097.2700,0,0,0,...,1,0,0,0,0,0,1,0,0,23
3,19.200,1,0.000000,182.0950,1.0,2,732.3800,0,0,0,...,0,0,0,0,0,0,0,0,0,24
4,8.930,0,0.000000,53.8614,2.0,2,994.7052,1,0,0,...,0,0,0,0,0,0,1,0,0,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,214.5218,2.0,2,2778.3834,1,0,0,...,0,0,0,1,0,0,1,0,0,35
8519,8.380,1,0.046982,108.1570,1.0,1,549.2850,0,0,0,...,0,0,0,0,0,0,1,0,0,20
8520,10.600,0,0.035186,85.1224,0.0,1,1193.1136,0,0,0,...,0,0,0,0,0,0,1,0,0,18
8521,7.210,1,0.145221,103.1332,1.0,2,1845.5976,0,0,1,...,0,0,0,1,0,0,0,1,0,13


In [37]:
x = df.drop('Item_Outlet_Sales', axis = 1)
y = df['Item_Outlet_Sales']

# MinMaxSclaer

In [38]:
nor_scaler = MinMaxScaler()
x_nor_df = nor_scaler.fit_transform(x)
x_df = pd.DataFrame(x_nor_df, columns = x.columns)
x_df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Est_Year
0,0.282525,0.0,0.048866,0.927507,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.416667
1,0.081274,1.0,0.058705,0.072068,0.5,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000
2,0.770765,0.0,0.051037,0.468288,0.5,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.416667
3,0.871986,1.0,0.000000,0.640093,0.5,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.458333
4,0.260494,0.0,0.000000,0.095805,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.916667


# Train Test Split 

In [39]:
x_train,x_test,y_train,y_test = train_test_split(x_df,y,test_size = 0.2, random_state = 1)
x_train

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Identifier_OUT013,Outlet_Identifier_OUT017,Outlet_Identifier_OUT018,Outlet_Identifier_OUT019,...,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Est_Year
1945,0.821375,0.0,0.272069,0.681925,0.5,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
1720,0.761834,0.0,0.511783,0.615374,0.5,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
1954,0.330158,1.0,0.164095,0.824788,0.5,0.5,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.083333
1919,0.374814,0.0,0.496066,0.312702,0.5,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.000000
2461,0.155701,1.0,0.181846,0.423207,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.208333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,0.479012,1.0,0.844906,0.531008,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
7813,0.479012,0.0,0.542624,0.100619,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
905,0.791605,0.0,0.200402,0.500067,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.500000
5192,0.300387,0.0,0.151094,0.054833,0.5,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.458333


# Train Model 

In [40]:
knn_model = KNeighborsRegressor(n_neighbors=7, p = 2)
knn_model.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=7)

In [41]:
y_pred = knn_model.predict(x_test)
y_pred[:5]

array([ 738.3722    , 1226.87917143, 2690.4978    ,  849.27545714,
       1063.75817143])

In [42]:
y_test[:5]

1070     952.7598
6305    1133.8574
8504    4138.6128
5562    1657.1762
1410     679.1160
Name: Item_Outlet_Sales, dtype: float64

# Evaluation

In [43]:
mse = mean_squared_error(y_test,y_pred)
mse

1558120.67201407

In [44]:
r2_score(y_test,y_pred)

0.4743747153136938

In [45]:
rmse = np.sqrt(mse)
rmse

1248.2470396576432

In [46]:
knn_model.score(x_test,y_test)

0.4743747153136938

# Hyperparameter tunning

In [47]:
k = np.arange(2,20)
p = [1,2]
hyp = {'n_neighbors' : k,
      'p' : p}

In [48]:
knn = KNeighborsRegressor()
best_model = GridSearchCV(knn, hyp, cv = 5)
best_model.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                         'p': [1, 2]})

In [49]:
best_model.best_params_

{'n_neighbors': 7, 'p': 2}

In [50]:
knn_model = KNeighborsRegressor(n_neighbors=20, p = 2)
knn_model.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=20)

In [51]:
y_pred = knn_model.predict(x_test)
y_pred[:5]

array([1571.35458, 1346.14773, 3386.5917 , 1705.21367, 1540.19514])

In [52]:
y_test

1070     952.7598
6305    1133.8574
8504    4138.6128
5562    1657.1762
1410     679.1160
          ...    
376     5715.2272
7708    4832.3764
3812    2972.1312
3928    2492.7552
7654    1717.7640
Name: Item_Outlet_Sales, Length: 1705, dtype: float64

In [53]:
x = r2_score(y_test,y_pred)
x

0.43360319655285573

In [60]:
knn_model.score(x_test,y_test)

0.43360319655285573